In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import joblib  # Para cargar predicciones crudas

# Suprimir warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import joblib  # Importar joblib directamente
import xgboost as xgb

# 1. Cargar el modelo entrenado
model_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/metadata_model.pkl'  # Reemplaza con la ruta de tu modelo
modelo = joblib.load(model_path)  # Cargar el modelo entrenado

# 2. Cargar el dataset de entrada (con los metadatos que usarás para predecir)

df = pd.read_csv('/Users/luiseduardogarciablanco/Desktop/nueva data cancer/2024/df_train_processed.csv')

/var/folders/sc/w6cv9_291mqc4c1m0qn06_4m0000gn/T/ipykernel_70449/3094498416.py:11: DtypeWarning: Columns (49,50,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/luiseduardogarciablanco/Desktop/nueva data cancer/2024/df_train_processed.csv')


In [4]:
# Lista de columnas a eliminar
columns_to_drop = ['Unnamed: 0', 'patient_id', 'sex', 'anatom_site_general', 'image_type', 'tbp_tile_type', 
                   'tbp_lv_location', 'tbp_lv_location_simple', 'attribution', 'copyright_license', 
                   'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5', 
                   'mel_mitotic_index', 'mel_thick_mm', 'tbp_lv_dnn_lesion_confidence', 
                   'combined_anatomical_site']

# Eliminar las columnas del DataFrame
df = df.drop(columns=columns_to_drop)

# Sumar los valores de todas las columnas restantes
column_sums = df.sum()

# Mostrar la suma de cada columna
print("Suma de los valores en cada columna después de eliminar las especificadas:")
print(column_sums)

Suma de los valores en cada columna después de eliminar las especificadas:
isic_id                   ISIC_0015670ISIC_0015845ISIC_0015864ISIC_00159...
target                                                                  389
age_approx                                                       22744720.0
clin_size_long_diam_mm                                           1543138.72
tbp_lv_A                                                     7834635.994465
                                                ...                        
onehot_38                                                              7976
onehot_39                                                             48223
onehot_40                                                            127915
onehot_41                                                             63003
onehot_42                                                             12638
Length: 274, dtype: object


In [5]:


# Asegúrate de que 'X' sea el conjunto de características sobre las cuales el modelo predice
X = df.drop(columns=['isic_id', 'target'])  # Eliminar columnas no relevantes para la predicción

# 3. Hacer predicciones crudas (probabilidades)
predicciones_crudas = modelo.predict_proba(X)  # Obtenemos las probabilidades

# Si es un problema binario, puedes querer la probabilidad solo de la clase positiva
# (asumiendo que la clase positiva es la segunda columna de la matriz de probabilidades)
probabilidades_clase_positiva = predicciones_crudas[:, 1]

# 4. Crear un nuevo dataframe con 'isic_id', 'target' y las predicciones crudas
df_resultados = df[['isic_id', 'target']].copy()  # Copiar columnas 'isic_id' y 'target'
df_resultados['prediccion_cruda'] = probabilidades_clase_positiva  # Añadir columna con las probabilidades crudas

# 5. Guardar el nuevo dataset con las predicciones crudas
output_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/data/nuevo_dataset_con_predicciones_2024.csv'  # Define dónde guardar el nuevo archivo
df_resultados.to_csv(output_path, index=False)

print(f"El archivo con las predicciones crudas ha sido guardado en {output_path}")

El archivo con las predicciones crudas ha sido guardado en /Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/data/nuevo_dataset_con_predicciones_2024.csv


Haremos una nueva prueba de entrenamiento del modelo de imagenes resnet50 añadiendo la nueva columna como caracteristica

In [19]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import joblib  # Para cargar predicciones crudas

# Suprimir warnings
import warnings
warnings.filterwarnings('ignore')

# Ruta donde se encuentran las imágenes y el archivo con predicciones crudas
image_path = '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/2024/Filtered_images'

# Cargar los metadatos
metadata_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/data/nuevo_dataset_con_predicciones_2024.csv'
metadata = pd.read_csv(metadata_path)


In [20]:
#comprobacion de imagenes en la carpeta

import os
import pandas as pd

# Obtener la lista de archivos en el directorio de imágenes
image_files = set(os.path.splitext(f)[0] for f in os.listdir(image_path) if f.lower().endswith('.jpg'))

# Obtener la lista de isic_id del DataFrame
isic_ids = set(metadata['isic_id'])

# Verificar qué imágenes están faltando
missing_images = isic_ids - image_files
if missing_images:
    print(f"Faltan las siguientes imágenes:")
    for img_id in missing_images:
        print(f"- {img_id}.jpg")
else:
    print("Todas las imágenes están presentes en la carpeta.")


Todas las imágenes están presentes en la carpeta.


In [21]:
import tensorflow as tf
# Listar los dispositivos físicos disponibles, en este caso, GPUs
physical_devices = tf.config.list_physical_devices('GPU')

# Si hay alguna GPU disponible
if len(physical_devices) > 0:
    # Permitir que TensorFlow crezca dinámicamente la memoria utilizada en la GPU
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

physical_devices

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [22]:
# Convertir la columna 'target' a string
metadata['target'] = metadata['target'].astype(str)

# Dividir los datos en entrenamiento y validación
train_df, val_df = train_test_split(metadata, test_size=0.2, stratify=metadata['target'], random_state=42)

print(train_df.shape)
print(val_df.shape)

(305524, 3)
(76382, 3)


In [23]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

# Generador personalizado para combinar imágenes y predicción cruda
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, image_dir, batch_size, target_size, shuffle=True, mode='train'):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.mode = mode  # Puede ser 'train' o 'val'
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_data = self.dataframe.iloc[index * self.batch_size:(index + 1) * self.batch_size]
        images = np.array([self._load_image(file_name) for file_name in batch_data['isic_id']])
        predicciones_crudas = np.array(batch_data['prediccion_cruda']).reshape(-1, 1)
        labels = np.array(batch_data['target']).astype('float32').reshape(-1, 1)

        if self.mode == 'train':
            return [images, predicciones_crudas], labels
        else:
            return [images, predicciones_crudas]

    def on_epoch_end(self):
        if self.shuffle:
            self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)

    def _load_image(self, image_name):
        img_path = os.path.join(self.image_dir, image_name + '.jpg')
        img = load_img(img_path, target_size=self.target_size)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)  # Aplicar normalización específica de ResNet50
        return img_array

# Crear los generadores de entrenamiento y validación
train_gen = CustomDataGenerator(train_df, image_dir=image_path, batch_size=32, target_size=(256, 256), shuffle=True)
val_gen = CustomDataGenerator(val_df, image_dir=image_path, batch_size=32, target_size=(256, 256), shuffle=False)

In [24]:
from tensorflow.keras.regularizers import l1_l2

# Cargar el modelo preentrenado ResNet50 (sin la parte superior)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Entrada de la imagen
image_input = layers.Input(shape=(256, 256, 3), name='image_input')
x = base_model(image_input)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Entrada de la predicción cruda
prediccion_cruda_input = layers.Input(shape=(1,), name='prediccion_cruda_input')

# Concatenar ambas entradas
combined = layers.Concatenate()([x, prediccion_cruda_input])

# Capa de salida
output = layers.Dense(1, activation='sigmoid', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(combined)

# Definir el modelo
model = models.Model(inputs=[image_input, prediccion_cruda_input], outputs=output)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), tf.keras.metrics.AUC(name='auc')])

# Resumen del modelo
model.summary()

2024-09-06 20:05:43.446921: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-09-06 20:05:43.446951: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-09-06 20:05:43.446956: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-09-06 20:05:43.447232: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-06 20:05:43.447257: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 resnet50 (Functional)       (None, 8, 8, 2048)           2358771   ['image_input[0][0]']         
                                                          2                                       
                                                                                                  
 flatten (Flatten)           (None, 131072)               0         ['resnet50[0][0]']            
                                                                                                  
 dense (Dense)               (None, 128)                  1677734   ['flatten[0][0]']         

In [25]:
# Entrenamiento inicial (solo las capas superiores)
history = model.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen
)

# Descongelar algunas capas del modelo base para fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Compilar de nuevo con una tasa de aprendizaje más baja
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), tf.keras.metrics.AUC(name='auc')])

# Calcular los pesos de clase
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_df['target']), y=train_df['target'])
class_weights = dict(enumerate(class_weights))

# Entrenamiento con Fine-Tuning
history_fine = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

Epoch 1/5


2024-09-06 20:05:53.989851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9547/9547 [==============================] - ETA: 0s - loss: 0.0136 - accuracy: 0.9988 - precision: 0.0128 - recall: 0.0033 - auc: 0.5653

2024-09-06 20:36:13.410468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9547/9547 [==============================] - 4545s 475ms/step - loss: 0.0136 - accuracy: 0.9988 - precision: 0.0128 - recall: 0.0033 - auc: 0.5653 - val_loss: 0.0109 - val_accuracy: 0.9990 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5328
Epoch 2/5
9547/9547 [==============================] - 6384s 669ms/step - loss: 0.0138 - accuracy: 0.9987 - precision: 0.0784 - recall: 0.0263 - auc: 0.5931 - val_loss: 0.0105 - val_accuracy: 0.9990 - val_precision: 1.0000 - val_recall: 0.0132 - val_auc: 0.5386
Epoch 3/5
9547/9547 [==============================] - 2055s 215ms/step - loss: 0.0132 - accuracy: 0.9987 - precision: 0.0962 - recall: 0.0329 - auc: 0.6116 - val_loss: 0.0109 - val_accuracy: 0.9990 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5454
Epoch 4/5
9547/9547 [==============================] - 2042s 214ms/step - loss: 0.0130 - accuracy: 0.9987 - precision: 0.1140 - recall: 0.0428 - auc: 0.6111 - val_loss: 0.0106 - val_accuracy: 0.9990 - val_pre

Epoch 1/10


2024-09-07 00:50:24.195485: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9547/9547 [==============================] - ETA: 0s - loss: 4.5245 - accuracy: 0.8516 - precision_1: 0.0030 - recall_1: 0.4507 - auc: 0.7178

2024-09-07 01:22:33.651647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9547/9547 [==============================] - 2324s 243ms/step - loss: 4.5245 - accuracy: 0.8516 - precision_1: 0.0030 - recall_1: 0.4507 - auc: 0.7178 - val_loss: 0.3023 - val_accuracy: 0.9022 - val_precision_1: 0.0055 - val_recall_1: 0.5395 - val_auc: 0.8303
Epoch 2/10
9547/9547 [==============================] - 2319s 243ms/step - loss: 1.6818 - accuracy: 0.7901 - precision_1: 0.0036 - recall_1: 0.7664 - auc: 0.8401 - val_loss: 0.5072 - val_accuracy: 0.9012 - val_precision_1: 0.0056 - val_recall_1: 0.5526 - val_auc: 0.8006
Epoch 3/10
9547/9547 [==============================] - 2320s 243ms/step - loss: 1.3801 - accuracy: 0.8108 - precision_1: 0.0042 - recall_1: 0.8092 - auc: 0.8638 - val_loss: 0.5833 - val_accuracy: 0.8493 - val_precision_1: 0.0039 - val_recall_1: 0.5921 - val_auc: 0.7810
Epoch 4/10
9547/9547 [==============================] - 2319s 243ms/step - loss: 1.0838 - accuracy: 0.8497 - precision_1: 0.0056 - recall_1: 0.8421 - auc: 0.8918 - val_loss: 0.4829 - val_accuracy: 0

In [26]:
import numpy as np
from sklearn.metrics import classification_report

# Función para recoger todas las etiquetas verdaderas del generador
def get_all_labels(generator):
    labels = []
    for i in range(len(generator)):
        _, batch_labels = generator[i]
        labels.extend(batch_labels)
    return np.array(labels).flatten()

# Evaluar el modelo
loss, accuracy, precision, recall, auc = model.evaluate(val_gen)
print(f"Loss: {loss}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUC: {auc}")

# Predecir las probabilidades
y_pred_probs = model.predict(val_gen)

# Ajustar el umbral de decisión
threshold = 0.5
y_pred_adjusted = (y_pred_probs > threshold).astype(int).flatten()

# Obtener las etiquetas verdaderas desde el generador
y_true = get_all_labels(val_gen)

# Asegurarse de que el número de predicciones coincida con el número de etiquetas verdaderas
assert len(y_pred_adjusted) == len(y_true), "El número de predicciones no coincide con el número de etiquetas reales."

# Calcular y mostrar el reporte de clasificación
print(classification_report(y_true, y_pred_adjusted, target_names=['Class 0', 'Class 1']))

2386/2386 [==============================] - 403s 169ms/step - loss: 0.3023 - accuracy: 0.9022 - precision_1: 0.0055 - recall_1: 0.5395 - auc: 0.8303
Loss: 0.3022826611995697, Accuracy: 0.9021898508071899, Precision: 0.0054856836795806885, Recall: 0.5394737124443054, AUC: 0.8303436636924744


2024-09-07 08:07:40.330648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2386/2386 [==============================] - 381s 159ms/step
              precision    recall  f1-score   support

     Class 0       1.00      0.90      0.95     76276
     Class 1       0.01      0.54      0.01        76

    accuracy                           0.90     76352
   macro avg       0.50      0.72      0.48     76352
weighted avg       1.00      0.90      0.95     76352



salida del modelo entrenado con la metadata total de 2024 con el añadido de la columna de la prediccion del primer modelo

2386/2386 [==============================] - 381s 159ms/step
              precision    recall  f1-score   support

     Class 0       1.00      0.90      0.95     76276
     Class 1       0.01      0.54      0.01        76

    accuracy                           0.90     76352
   macro avg       0.50      0.72      0.48     76352
weighted avg       1.00      0.90      0.95     76352



In [27]:
# Ruta donde guardar el modelo
model_save_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/prueba_modelo_encadenado.pkl'

# Guardar el modelo
model.save(model_save_path)
print(f"Modelo guardado en {model_save_path}")

INFO:tensorflow:Assets written to: /Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/prueba_modelo_encadenado.pkl/assets


INFO:tensorflow:Assets written to: /Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/prueba_modelo_encadenado.pkl/assets


Modelo guardado en /Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/prueba_modelo_encadenado.pkl
